In [2]:
# RUN THIS COMMAND ONLY IF YOU USE GOOGLE COLAB.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Colab_Notebooks/TechLabs/MP

/content/drive/MyDrive/Colab_Notebooks/TechLabs/MP


**Loading the Data from Versammlungsbehörde Berlin**

In [4]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100) # Show all columns when looking at dataframe

In [6]:
df = pd.read_json("data/versammlungsbehörde_berlin_2021.json", lines=True  ) ## = True

ValueError: Expected object or value

ValueErrot: Expected object or value



In [ ]:
df.head()

,0,1
0,payload,"{'allShortcutsEnabled': False, 'fileTree': {'d..."
1,title,berlin/data/jsonl/20210628-180744-7c81b6113d9f...


In [ ]:
# get columns names
col_names = df.columns
col_names

RangeIndex(start=0, stop=2, step=1)

In [ ]:
rows = df.shape[0]
columns = df.shape[1]
print("Our survey Data has {} rows and {} columns.".format(rows, columns))

Our survey Data has 2 rows and 2 columns.


-----------
---------
-----------
##Scraping
trying out code to make this work easier:


In [ ]:
import re
zwei_tage_im_juli = '''
[{"id": "46", "start": "2021-07-02T17:45:00", "end": "2021-07-02T21:00:00", "topic": "Video-Aktion zur Aufkl\u00e4rung \u00fcber die Tierindustrie", "zip": "12435", "place": "Treptower Park", "route": "", "lfdnr": "16"}
{"id": "49", "start": "2021-07-02T19:00:00", "end": "2021-07-02T21:00:00", "topic": "Kundgebung als Protest zur Freilassung syrischer  politischer Gefangene.", "zip": "10117", "place": "Pariser Platz", "route": "", "lfdnr": "17"}
{"id": "52", "start": "2021-07-02T19:00:00", "end": "2021-07-02T21:00:00", "topic": "Keine Drogen. Nein zu Gewalt.", "zip": "10785", "place": "Potsdamer Platz", "route": "", "lfdnr": "18"}
{"id": "55", "start": "2021-07-02T19:00:00", "end": "2021-07-02T23:00:00", "topic": "Videokundgebung: von der Krise zur Enteignung ", "zip": "13347", "place": "Leopoldplatz", "route": "", "lfdnr": "19"}
{"id": "1", "start": "2021-07-03T10:00:00", "end": "2021-07-03T12:00:00", "topic": "Mahnwache: Aufhebung der Corona-Zwangsma\u00dfnahmen. Respekt f\u00fcr die Grundrechte!", "zip": "14195", "place": "K\u00f6nigin-Luise-Str.", "route": "", "lfdnr": "1"}
{"id": "4", "start": "2021-07-03T10:00:00", "end": "2021-07-03T12:00:00", "topic": "\u201eLandespolitik in Bezug auf die Abgeordnetenhauswahl am 26.09.2021\u201c", "zip": "10245", "place": "Boxhagener Platz", "route": "", "lfdnr": "2"}
{"id": "7", "start": "2021-07-03T10:00:00", "end": "2021-07-03T18:00:00", "topic": "Gegen Mieterverdr\u00e4ngung durch sogenannte energetische Sanierung und Luxussanierung f\u00fcr unverf\u00e4lschte Denkmalpflege. Versto\u00df gegen den Kaufvertrag. (vom 20.06. bis 19.09.2021 - t\u00e4glich)", "zip": "13509", "place": "An der Heide 1", "route": "An der Heide Ecke Kehrwieder und Am Brunnen Ecke am Rosensteg  (alle 2 Stunden im Wechsel)  ", "lfdnr": "3"}
{"id": "10", "start": "2021-07-03T11:00:00", "end": "2021-07-03T12:00:00", "topic": "Mahnwache f\u00fcr Toleranz und ein friedliches Miteinander", "zip": "13597", "place": "Marktstr.", "route": "", "lfdnr": "4"}
{"id": "13", "start": "2021-07-03T11:00:00", "end": "2021-07-03T13:00:00", "topic": "Arbeitslosigkeit und Wirtschaftskrise", "zip": "12679", "place": "Marzahner Promenade 1A", "route": "", "lfdnr": "5"}
{"id": "16", "start": "2021-07-03T11:00:00", "end": "2021-07-03T14:00:00", "topic": "Mahnwache: Verh\u00e4ltnism\u00e4\u00dfigkeit der Coronama\u00dfnahmen (vom 03.07. bis 25.09.2021 - jeweils Sa.)", "zip": "14163", "place": "Kirchstr.", "route": "", "lfdnr": "6"}
{"id": "19", "start": "2021-07-03T11:30:00", "end": "2021-07-03T16:30:00", "topic": "B\u00e4ume Bew\u00e4sserungen gegen die Personalnot im  Neuk\u00f6llner Gr\u00fcnfl\u00e4chenamt", "zip": "", "place": "", "route": "Gr\u00fcnmittelstreifen Blaschkoalle - Britzer Damm - Parchimer Allee/Onkel Br\u00e4sig Str.", "lfdnr": "7"}
{"id": "22", "start": "2021-07-03T12:00:00", "end": "2021-07-03T13:40:00", "topic": "Brasilianisches Gesundheitsministerium zu Korruption beim Kauf von Covaxin-Impfstoffen - Raus Bolsonaro/Mauro und all Genozid", "zip": "10117", "place": "Pariser Platz", "route": "", "lfdnr": "8"}
{"id": "25", "start": "2021-07-03T12:00:00", "end": "2021-07-03T14:30:00", "topic": "Stoppt die Verfolgung von FALUN DAFA / FALUN-GONG Praktizierenden in China. Beendet die Diktatur der Kommunistischen Partei. (vom 02.01. bis 31.12.2021 - jeweils Mo., Di., Mi., Do., Fr., Sa., So.)", "zip": "10179", "place": "Jannowitzbr\u00fccke", "route": "", "lfdnr": "9"}
{"id": "28", "start": "2021-07-03T12:00:00", "end": "2021-07-03T17:00:00", "topic": "Team Todenh\u00f6fer - Die Gerechtigkeitspartei setzt sich ein f\u00fcr mehr Gerechtigkeit in Deutschland, dabei wollen wir vor allem mehr Generationen-Gerechtigkeit, mehr Bildungs-Gerechtigkeit, mehr Steuer.Gerechtigkeit", "zip": "10967", "place": "Hasenheide", "route": "", "lfdnr": "10"}
{"id": "31", "start": "2021-07-03T12:00:00", "end": "2021-07-03T18:00:00", "topic": "Freiheit ist nicht Verhandelbar, f\u00fcr Souver\u00e4nit\u00e4t", "zip": "10117", "place": "Platz des 18. M\u00e4rz", "route": "", "lfdnr": "11"}
{"id": "34", "start": "2021-07-03T12:00:00", "end": "2021-07-03T18:00:00", "topic": "Performative k\u00fcnstlerische Lesung und Aktion im  \u00f6ffentlichen Raum entitelt FEMINISMUS VOR ORT wird  von 9 Kunsterinnen unternomen als Teil des Programms  Drau\u00dfenstadt des Kulturamts Steglitz-Zehlendorf.", "zip": "", "place": "", "route": "Stadtpark Steglitz: Albrechtstr./Bismarkstr. -  Weverpromenade - Font\u00e4nenparkteig - Stadtparkteig -  rund um Rosengarten", "lfdnr": "12"}
{"id": "37", "start": "2021-07-03T12:00:00", "end": "2021-07-03T18:00:00", "topic": "F\u00fcr mehr Transparenz, Sachlichkeit und  Verh\u00e4ltnism\u00e4\u00dfigkeit in der Corona-Politik und den Medien", "zip": "10178", "place": "Alexanderplatz", "route": "", "lfdnr": "13"}
{"id": "40", "start": "2021-07-03T13:00:00", "end": "2021-07-03T16:00:00", "topic": "Start der ersten Phase des Volksbegehrens Berlin 2030 klimaneutral zur \u00c4nderung des Berliner  Energiewendege- setzes und Anpassung an die  Vereinbarungen des Pariser Klimaschutzabkommens.", "zip": "10967", "place": "Hasenheide 17", "route": "", "lfdnr": "14"}
{"id": "43", "start": "2021-07-03T13:00:00", "end": "2021-07-03T17:00:00", "topic": "Demo gegen Milit\u00e4rputsch in Myanmar", "zip": "14169", "place": "Katharinenstr. 31", "route": "", "lfdnr": "15"}
{"id": "46", "start": "2021-07-03T13:00:00", "end": "2021-07-03T17:00:00", "topic": "Zum Jahresgedenken zur Ermordung des Oromo-Aktivisten  Hacaaluu Hundeessaa am 29.06.2020 und im Gedenken an alle anderen Ermordeten Oromos durch die \u00e4thiopische  Regierung", "zip": "10178", "place": "Alexanderplatz 2", "route": "", "lfdnr": "16"}
{"id": "49", "start": "2021-07-03T13:00:00", "end": "2021-07-03T17:00:00", "topic": "Mahnwache - Gegen Totalitarismus - Gedenken an Stefan Lux anl\u00e4sslich seines 85. Todestages", "zip": "10117", "place": "Unter den Linden 77", "route": "", "lfdnr": "17"}
{"id": "52", "start": "2021-07-03T13:00:00", "end": "2021-07-03T19:00:00", "topic": "Solidarit\u00e4t statt Kapitalismus", "zip": "10585", "place": "Wilmersdorfer Str.", "route": "", "lfdnr": "18"}
{"id": "55", "start": "2021-07-03T13:00:00", "end": "2021-07-03T20:00:00", "topic": "Stoppt die Verfolgung von FALUN DAFA / FALUN-GONGPraktizierenden und anderen unschuldigen Menschen in China. Beendet die Diktatur der Kommunistischen Partei.", "zip": "10407", "place": "Am Friedrichshain", "route": "", "lfdnr": "19"}
{"id": "58", "start": "2021-07-03T13:30:00", "end": "2021-07-03T19:00:00", "topic": "Aufmerksamkeit f\u00fcr Kultur auf dem Wasser", "zip": "10245", "place": "Rummelsburger See 24B", "route": "Halbinsel Stralau; Uferweg (Bartholom\u00e4us- Ufer)", "lfdnr": "20"}
{"id": "61", "start": "2021-07-03T14:00:00", "end": "2021-07-03T16:00:00", "topic": "Bundesweite Mahnwachen vor Tierversuchslaboren. Nein zu Tierversuchen!", "zip": "13353", "place": "M\u00fcllerstr. 178", "route": "", "lfdnr": "21"}
{"id": "64", "start": "2021-07-03T14:00:00", "end": "2021-07-03T16:00:00", "topic": "NEUE NORMALIT\u00c4T ABLEHNEN Neuerdings m\u00fcssen wir politisch Passiven die erste und letzte M\u00f6glickeit demokratischer Teilhabe durch das ersammlungsrecht nutzen, um darauf aufmerksam zu machen, dass dieses untypische Hygienedenken und - verhalten uns allen schlecht zu Gesicht steht. Wir fordern f\u00fcr alle freien Menschen, f\u00fcr Alt und Jung, f\u00fcr Arm und  Reich, f\u00fcr Rechts und Links, die R\u00fcckkehr zu freiheitlich demokratischen Lebensweisen im Sinne einer offenen, pluralistischen Gesellschaft, in der es das Individuum zu einer freien Entfaltung seiner Selbst bringen darf, und zwar in jeglicher Hinsicht, sofern damit nicht jemand Anderes direkt gesch\u00e4digt wird. Der simple Fakt, dass Menschen atmen, darf kein  hinreichender Grund daf\u00fcr sein, Alle pauschal und  dauerhaft in ihren verfassungsgem\u00e4\u00dfen reiheitsrechten einzuschr\u00e4nken.", "zip": "10437", "place": "Schwedter Str. ", "route": "", "lfdnr": "22"}
{"id": "67", "start": "2021-07-03T14:00:00", "end": "2021-07-03T17:00:00", "topic": "Freiheit ist nicht verhandelbar.", "zip": "", "place": "", "route": "neu:  Platz des 18. M\u00e4rz - Ebertstr. - Dorotheenstr. - Wilhelmstr. - Unter den Linden - Unter den Linden 63-65 an der russischen Botschaft (ZK) - Unter den Linden - Wilhelmstr. - Dorotheenstr. - Scheidemannstr. - Heinrich-von-Gagern-Str. Ecke Paul-L\u00f6be-Allee nahe Bundeskanzleramt (ZK) - Heinrich-von-Gagern-Str. - Scheidemannstr. -Ebertstr. - Platz des 18. M\u00e4rz (EK)    alt:  Platz des 18. M\u00e4rz - Unter den Linden - Unter den Linden 63-65 an der russischen Botschaft (ZK) - Unter den Linden - Wilhelmstr. - Dorotheenstr. - Scheidemannstr. - Heinrich-von-Gagern-Str. Ecke Paul-L\u00f6be-Allee nahe Bundeskanzleramt (ZK) - Heinrich-von-Gagern-Str. - Scheidemannstr. -Ebertstr. - Platz des 18. M\u00e4rz (EK)", "lfdnr": "23"}
{"id": "70", "start": "2021-07-03T14:00:00", "end": "2021-07-03T17:30:00", "topic": "Kein US-Krieg gegen Venezuela, USA Unblock Venezuela, USA Unblock Kuba, Gegen Intervention und Blockade, H\u00e4nde weg von dem fortschrittlichen Lateinamerika (vom 22.05. bis 14.08.2021 - jeweils Sa.)", "zip": "10117", "place": "Pariser Platz", "route": "", "lfdnr": "24"}
{"id": "73", "start": "2021-07-03T15:00:00", "end": "2021-07-03T16:00:00", "topic": "Freiheit f\u00fcr Ali Osman K\u00f6se", "zip": "10999", "place": "Oranienplatz 1", "route": "", "lfdnr": "25"}
{"id": "76", "start": "2021-07-03T15:00:00", "end": "2021-07-03T17:00:00", "topic": "Heimat und Weltfrieden (vom 06.02. bis 31.12.2021 - jeweils Sa.)", "zip": "10557", "place": "Platz der Republik 1", "route": "", "lfdnr": "26"}
{"id": "79", "start": "2021-07-03T15:00:00", "end": "2021-07-03T17:00:00", "topic": "Wir wollen eine kleine Aktion machen, um Aufmerksamkeit darauf zu lenken, dass mit Julian Assange ein Journalist in Gro\u00dfbritannien im Gef\u00e4ngnis sitzt und droht an die USA ausgeliefert zu werden. Wir wollen uns f\u00fcr Pressefreiheit einsetzen!", "zip": "10117", "place": "Pariser Platz", "route": "", "lfdnr": "27"}
{"id": "82", "start": "2021-07-03T15:00:00", "end": "2021-07-03T21:30:00", "topic": "Geradedenken//Rave \u2013 Demo-Rave gegen  Verschw\u00f6rungsideo- logie und rechtsoffene Rave-Events", "zip": "12099", "place": "Tempelhofer Feld", "route": "", "lfdnr": "28"}
{"id": "85", "start": "2021-07-03T15:00:00", "end": "2021-07-03T22:00:00", "topic": "F\u00f6rderung der Basisdemokratie", "zip": "12627", "place": "Alice-Salomon-Platz", "route": "(Neu)  Theaterplatz - Louis-Lewin-Str. - Quedlinburger Str. - Tangerm\u00fcnder Str. - Janusz-Korczak-Str. - Senftenberger Str. - Kastanienallee (ZK) - Cottbusser Str. - Gothaer Str. - Kyritzer Str. - Alte Hellersdorfer Str. - Zossener Str. - Klausdorfer Str. - Zossener Str. - Teupitzer Str. - Stendaler Str. -  Janusz-Korczak-Str. - Stendaler Str. - Alice-Salomon-Platz   (Alt)  Theaterplatz - Louis-Lewin-Str. - Quedlinburger Str. - Stendaler Str. - Janusz-Korczak-Str. - Senftenberger Str. - Kastanienallee (ZK) - Cottbusser Str. - Gothaer Str. - Kyritzer Str. - Alte Hellersdorfer Str. - Zossener Str. - Klausdorfer Str. - Zossener Str. - Teupitzer Str. - Stendaler Str. - Janusz-Korczak-Str. - Stendaler Str. - Alice-Salomon-Platz", "lfdnr": "29"}
{"id": "88", "start": "2021-07-03T15:00:00", "end": "2021-07-03T23:59:00", "topic": "Antirepression", "zip": "10997", "place": "Mariannenplatz", "route": "", "lfdnr": "30"}
{"id": "91", "start": "2021-07-03T16:00:00", "end": "2021-07-03T18:00:00", "topic": "Der Wedding Enteignet  Politischer Abschluss der Sammelphase in Berlin Wedding  der Kampagne Deutsche Wohnen und Co enteignen.", "zip": "13353", "place": "M\u00fcllerstr. 147", "route": "", "lfdnr": "31"}
{"id": "94", "start": "2021-07-03T16:00:00", "end": "2021-07-03T19:00:00", "topic": "Dieses System macht uns Krank! - unsere Gesundheit statt ihrer Profite", "zip": "10967", "place": "Dieffenbachstr.", "route": "", "lfdnr": "32"}
{"id": "97", "start": "2021-07-03T17:00:00", "end": "2021-07-03T20:00:00", "topic": "FREIHEIT F\u00dcR JULIAN ASSANGE! Pressefreiheit,  Meinung- sfreiheit: Grundlage f\u00fcr Demokratie und Frieden.  Geburtstagskun- dgebung f\u00fcr Julian Assange.", "zip": "", "place": "", "route": "NEU  Pariser Platz (AK) - Unter den Linden - Unter den Linden/Wilhelmstr. (ZK Nahe der britischen Botschaft) - Unter den Linden - Pariser Platz (EK)    ALT  Pariser Platz (AP US Botschaft)- Unter den Linden-  Wilhelmstra\u00dfe  (ZK Britischen Botschaft)- Unter den Linden - Pariser Platz", "lfdnr": "33"}
{"id": "100", "start": "2021-07-03T17:00:00", "end": "2021-07-03T20:00:00", "topic": "Aktuelle brasilianische Situation bzw. Regierung Jair  Bolsonaro, Covid-19, Indigenen und Position von EU.", "zip": "10117", "place": "", "route": "Lustgarten (Humboldt Forum AK) - Unten den Linden -  Glinkastr. - Kronenstr. - Jerusalemer Str. - Hausvogteiplatz - Oberwallstr. - J\u00e4gerstr. - Kurstr. - Werderscher Markt (Ausw\u00e4rtiges Amt EK) ", "lfdnr": "34"}
{"id": "103", "start": "2021-07-03T18:00:00", "end": "2021-07-03T21:00:00", "topic": "Solidarit\u00e4t mit dem kurdischen Volk", "zip": "", "place": "", "route": "Hermannplatz - Sonnenallee - Pannierstr. -  Wiener Str. - Oranienstr. - Oranienplatz", "lfdnr": "35"}
{"id": "1", "start": "2021-07-04T10:00:00", "end": "2021-07-04T18:00:00", "topic": "Gegen Mieterverdr\u00e4ngung durch sogenannte energetische Sanierung und Luxussanierung f\u00fcr unverf\u00e4lschte Denkmalpflege. Versto\u00df gegen den Kaufvertrag. (vom 20.06. bis 19.09.2021 - t\u00e4glich)", "zip": "13509", "place": "An der Heide 1", "route": "An der Heide Ecke Kehrwieder und Am Brunnen Ecke am Rosensteg  (alle 2 Stunden im Wechsel)  ", "lfdnr": "1"}
{"id": "4", "start": "2021-07-04T12:00:00", "end": "2021-07-04T14:30:00", "topic": "Stoppt die Verfolgung von FALUN DAFA / FALUN-GONG Praktizierenden in China. Beendet die Diktatur der Kommunistischen Partei. (vom 02.01. bis 31.12.2021 - jeweils Mo., Di., Mi., Do., Fr., Sa., So.)", "zip": "10179", "place": "Jannowitzbr\u00fccke", "route": "", "lfdnr": "2"}
{"id": "7", "start": "2021-07-04T12:00:00", "end": "2021-07-04T14:30:00", "topic": "Epidemische Lage nationaler Tragweite???  Grundrechte gelten immer und f\u00fcr alle:  geimpft, getestet, genesen, gesund oder krank. (vom 04.07. bis 03.10.2021 - jeweils So.)", "zip": "14163", "place": "Fischerh\u00fcttenstr.", "route": "", "lfdnr": "3"}
{"id": "10", "start": "2021-07-04T12:00:00", "end": "2021-07-04T16:00:00", "topic": "Der Pflegenotstand und politische Verfehlung der CDU/kirchlicher Tr\u00e4ger , die in Widerspruch zu christlichen Werten stehen.", "zip": "10117", "place": "Pariser Platz", "route": "", "lfdnr": "4"}
{"id": "13", "start": "2021-07-04T12:00:00", "end": "2021-07-04T19:00:00", "topic": "Stoppt die Verfolgung von FALUN DAFA / FALUN-GONG Praktizierenden und anderen unschuldigen Menschen in China. Beendet die Diktatur der Kommunistischen Partei. (vom 10.01. bis 26.12.2021 - jeweils So.)", "zip": "12435", "place": "Treptower Park", "route": "", "lfdnr": "5"}
{"id": "16", "start": "2021-07-04T12:00:00", "end": "2021-07-04T19:00:00", "topic": "Multikulturelles Friedensforum, Podiumsdiskussion", "zip": "14053", "place": "Olympischer Platz", "route": "", "lfdnr": "6"}
{"id": "19", "start": "2021-07-04T12:00:00", "end": "2021-07-04T20:00:00", "topic": "Stoppt die Verfolgung von FALUN DAFA / FALUN-GONGPraktizierenden und anderen unschuldigen Menschen in China. Beendet die Diktatur der Kommunistischen Partei. (vom 23.05. bis 25.07.2021 - jeweils So.)", "zip": "10437", "place": "Mauerpark", "route": "", "lfdnr": "7"}
{"id": "22", "start": "2021-07-04T14:00:00", "end": "2021-07-04T17:00:00", "topic": "Unterst\u00fctzung der algerischen Revolution (vom 06.06. bis 26.12.2021 - jeweils So.)", "zip": "10117", "place": "Pariser Platz", "route": "", "lfdnr": "8"}
{"id": "25", "start": "2021-07-04T15:00:00", "end": "2021-07-04T17:00:00", "topic": "Eine Demo gegen die Belagerung der Stadt Daraa durch  Assad und Russland Militien", "zip": "10117", "place": "Unter den Linden", "route": "", "lfdnr": "9"}
{"id": "28", "start": "2021-07-04T15:00:00", "end": "2021-07-04T22:00:00", "topic": "Mahnwachen f\u00fcr den Frieden, f\u00fcr Gerechtigkeit, f\u00fcr Freiheit  \u2026", "zip": "", "place": "", "route": "NEU: (Stand 29.06.21)  Pariser Platz - Unter den Linden - Schadowstr. - Dorotheenstr. - Bunsenstr. - Reichstagsufer - ARD-Hauptstadtstudio (ZK) - Reichstagsufer - Neust\u00e4dtische Kirchstr. - Unter den Linden - ZDF Hauptstadtstudio (ZK) - Friedrichstr. - Weidendammer Br\u00fccke - Schiffbauerdamm - Berliner Ensemble (ZK) - Schiffbauerdamm - Weidendammer Br\u00fccke - Am Weidendamm - Geschwister-Scholl-Str. - Universit\u00e4tsstr. - Unter den Linden - Bebelplatz (ZK) - Unter den Linden - Am Lustgarten    ALT:  Br- andenburger Tor - \u2026. - Platz der Republik - \u2026. - Reichstagufer / Wilhelmstr. - \u2026.  - Museumsinsel - Lustgarten", "lfdnr": "10"}
{"id": "31", "start": "2021-07-04T17:00:00", "end": "2021-07-04T21:00:00", "topic": "Frieden, Freiheit, Demokratie-Analoger Meinungsaustausch-Offener Debattenraum", "zip": "14089", "place": "Imchenallee 36", "route": "", "lfdnr": "11"}
{"id": "34", "start": "2021-07-04T17:45:00", "end": "2021-07-04T21:00:00", "topic": "Video-Aktion zur Aufkl\u00e4rung \u00fcber die Tierindustrie", "zip": "12435", "place": "Treptower Park", "route": "", "lfdnr": "12"}]
'''

pattern = re.compile(r'}')
matches = pattern.finditer(zwei_tage_im_juli)
for match in matches:
  print(match)

file = open("zwei_tage_im_juli", "w")

for match in zwei_tage_im_juli:
    file.write(match)
    file.write(", ")

file.close()

print(zwei_tage_im_juli)

<re.Match object; span=(209, 210), match='}'>
<re.Match object; span=(439, 440), match='}'>
<re.Match object; span=(628, 629), match='}'>
<re.Match object; span=(831, 832), match='}'>
<re.Match object; span=(1069, 1070), match='}'>
<re.Match object; span=(1295, 1296), match='}'>
<re.Match object; span=(1728, 1729), match='}'>
<re.Match object; span=(1935, 1936), match='}'>
<re.Match object; span=(2138, 2139), match='}'>
<re.Match object; span=(2383, 2384), match='}'>
<re.Match object; span=(2675, 2676), match='}'>
<re.Match object; span=(2957, 2958), match='}'>
<re.Match object; span=(3312, 3313), match='}'>
<re.Match object; span=(3678, 3679), match='}'>
<re.Match object; span=(3890, 3891), match='}'>
<re.Match object; span=(4353, 4354), match='}'>
<re.Match object; span=(4608, 4609), match='}'>
<re.Match object; span=(4953, 4954), match='}'>
<re.Match object; span=(5150, 5151), match='}'>
<re.Match object; span=(5482, 5483), match='}'>
<re.Match object; span=(5736, 5737), match='}'>
